# 흐름 (괄호는 코스닥만)
1. 읽어들이기 
2. 테이블 다중인덱스로 만들기 --> 편하게 하기위해서 --> PEIS 랑 똑같은 데이터 프레임형태
2. 목표주가, 당시주가, 수정주가, 애널리스트등급, 시가총액 일자로 세워서 붙이기
3. 수정주가 기준 수익률 계산해주기
4. fillna 99 해서 수익률 없는거 99해주기 --> 나중에 지우기 쉽게하기위해 임의로 박아준거임
* (코스닥) - "스팩" 데이터 제거하기 
5. 목표주가랑 현재주가 기준 컨센 구하기 
6. PEIS랑 주가데이터랑 결합해서 저장

# 수익률 PEIS점수테이블에 붙이기 

In [76]:
import pandas as pd
import numpy as np
import warnings
from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 파일불러들이기 붙여질 파일(PEIS) + 붙일 파일(price)
# 목표 : 주가랑 주가상승률, 애널리스트등급 붙이기

In [77]:
PEIS_table = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kospi_peis")
PROFIT_table = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kospi_stock")

# 상폐되어 버려서 아예 없는 기업들 제거 _ 원컬럼으로 세웠을때 데이터 맞출려면 필수적

In [78]:
delistcom = set(PEIS_table['Name'].unique())-set(PROFIT_table['Symbol Name'].unique())
dropdata(PEIS_table, 'Name', delistcom)
Symbolindexcom = set(PEIS_table['Symbol'].unique())-set(PROFIT_table['Symbol'].unique())
dropdata(PEIS_table, 'Symbol', Symbolindexcom)

# 데이터 전처리 
# 1. 주가랑 주가상승률, 애널리스트등급 행렬로 일자로 만들기

In [80]:
PROFIT_table = MakeLookGood(PROFIT_table, years = list(range(10,22))) # 새로 뽑으면 10 -> 8
Onetable = MakeOnetable(PROFIT_table, ['수정주가(원)','투자의견점수 (E3)(포인트)'], ['ADJprice', 'Consensus'])
Onetable['deltaprice'] = Caldeltaprice(Onetable['ADJprice'])
# 여기서 deltaprice 수정사항 넣어야함
Onetable = Onetable.fillna(99)  

# 1.5 컨센 점수 가공

In [81]:
consen_cut = pd.cut(Onetable['Consensus'],bins= [-0.1, 0, 2.5, 3.5, 4.5, 5], labels = [0,1,2,3,4])
Onetable['consen_cut'] = consen_cut

In [82]:
CompuConsen(Onetable)

# 2. 붙이기

In [83]:
Onetable = pd.concat([PEIS_table,Onetable],axis=1)

# 3. 코스피도 스팩이 있음 

In [84]:
names = Onetable['Name']
specindex = returnspecindex(names)
dropdata(Onetable, 'Name', specindex)
#Onetable.drop(specindex,inplace = True)

In [85]:
Onetable

,Symbol,Name,회계년,총자산(천원),총자산(평균)(천원),영업이익(천원),매출원가(천원),판매비와관리비(천원),금융원가(비영업)(천원),기타비용(비영업)(천원),...,매출총이익(천원),영업활동으로인한현금흐름(천원),매출채권(천원),재고자산(천원),매입채무(천원),ADJprice,Consensus,deltaprice,consen_cut,CompuConsen
0,A005930,삼성전자,2010,1.342887e+11,1.232343e+11,1.662103e+10,1.026668e+11,3.534247e+10,7.700099e+09,1.079935e+09,...,5.196350e+10,2.382678e+10,1.915311e+10,1.336452e+10,9.148661e+09,18980.0,4.00,0.438563,3,4
1,A005930,삼성전자,2011,1.558003e+11,1.450445e+11,1.564429e+10,1.121451e+11,3.721236e+10,7.893421e+09,1.612690e+09,...,5.285665e+10,2.291790e+10,2.188213e+10,1.571672e+10,1.027673e+10,21160.0,4.00,-0.098555,3,4
2,A005930,삼성전자,2012,1.810716e+11,1.684359e+11,2.904934e+10,1.266519e+11,4.540234e+10,7.934450e+09,1.576025e+09,...,7.445168e+10,3.797281e+10,2.386124e+10,1.774741e+10,9.489111e+09,30440.0,4.00,-0.032799,3,4
3,A005930,삼성전자,2013,2.140750e+11,1.975733e+11,3.678501e+10,1.376963e+11,5.421134e+10,7.754972e+09,1.614048e+09,...,9.099636e+10,4.670744e+10,2.498853e+10,1.913487e+10,8.437139e+09,27440.0,3.96,-0.050490,3,4
4,A005930,삼성전자,2014,2.304230e+11,2.222490e+11,2.502507e+10,1.282788e+11,5.290212e+10,7.294002e+09,2.259737e+09,...,7.792719e+10,3.697539e+10,2.469461e+10,1.731750e+10,7.914704e+09,26540.0,3.96,0.430159,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10507,A004110,대선주조,2017,8.253397e+07,7.345201e+07,4.056389e+06,2.594335e+07,2.069631e+07,2.318230e+05,5.406970e+05,...,2.475270e+07,2.023088e+07,9.901398e+06,2.080131e+06,4.839368e+06,0.0,0.00,99.000000,0,5
10508,A004110,대선주조,2018,9.562874e+07,8.908136e+07,1.049800e+07,3.882492e+07,3.191513e+07,2.849760e+05,9.960970e+05,...,4.241312e+07,1.350382e+07,9.779005e+06,2.590797e+06,5.239875e+06,0.0,0.00,99.000000,0,5
10509,A004110,대선주조,2019,9.800334e+07,9.681604e+07,1.017116e+07,4.202774e+07,3.355528e+07,2.614470e+05,1.054431e+06,...,4.372645e+07,1.180832e+07,9.202183e+06,2.492168e+06,4.795998e+06,0.0,0.00,99.000000,0,5
10510,A004110,대선주조,2020,9.878299e+07,9.839316e+07,1.110870e+07,3.786030e+07,2.227974e+07,1.417550e+05,6.704810e+05,...,3.338844e+07,5.548303e+06,1.007206e+07,2.407586e+06,4.237584e+06,0.0,0.00,0.000000,0,5


# 4. 저장하기

In [86]:
# 이렇게 만든거는.. 일단 주가 정보가 없는 친구들도 있어서 얘들은 포트폴리오로 구성하기가 힘들 듯함 상장전인듯
# 그리고 중간에 상폐당한 친구들은 또 PEIS는 있는데 주가가 없어서 포폴게산이 안될떄가 있을 거 같음 우짬.. 
Onetable.to_excel("Data/KospiPEISData&PriceData.xlsx",index= False)

#

#

#


#

#

#

#

#

#


#

# 코스닥 똑같이 만들기
* 코스닥도 나중에 코스피랑 똑같은 전처리 과정에서 문제 없는지 확인해야함

In [87]:
PEIS_table_kosdaq = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kosdaq_peis")
PROFIT_table_kosdaq = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kosdaq_stock")

In [88]:
delistcom = set(PEIS_table_kosdaq['Name'].unique())-set(PROFIT_table_kosdaq['Symbol Name'].unique())
dropdata(PEIS_table_kosdaq, 'Name', delistcom)
Symbolindexcom = set(PEIS_table_kosdaq['Symbol'].unique())-set(PROFIT_table_kosdaq['Symbol'].unique())
dropdata(PEIS_table_kosdaq, 'Symbol', Symbolindexcom)

In [90]:
PROFIT_table_kosdaq = MakeLookGood(PROFIT_table_kosdaq, years = list(range(10,22)))
# 이 아래다가 목표주가-target_price, 시가총액-MarketValue
Onetable_kosdaq = MakeOnetable(PROFIT_table_kosdaq, ['수정주가(원)','투자의견점수 (E3)(포인트)'], ['ADJprice', 'Consensus'])
Onetable_kosdaq['deltaprice'] = Caldeltaprice(Onetable_kosdaq['ADJprice'])
# 여기서 deltaprice 수정사항 넣어야함
Onetable_kosdaq = Onetable_kosdaq.fillna(99)  
consen_cut = pd.cut(Onetable_kosdaq['Consensus'],bins= [-0.1, 0, 2.5, 3.5, 4.5, 5], labels = [0,1,2,3,4])
Onetable_kosdaq['consen_cut'] = consen_cut

In [92]:
CompuConsen(Onetable_kosdaq)

In [96]:
Onetable_kosdaq = pd.concat([PEIS_table_kosdaq,Onetable_kosdaq],axis=1)

# 코스닥 스팩빼기

In [97]:
names = Onetable_kosdaq['Name']
specindex = returnspecindex(names)
Onetable_kosdaq.drop(specindex,inplace = True)

In [98]:
Onetable_kosdaq.to_excel("Data/KosdaqPEISData&PriceData.xlsx",index= False)

# 나중에 넣을 것 예시

In [ ]:
Onetable_kosdaq = MakeOnetable(PROFIT_table_kosdaq, ['수정주가(원)','투자의견점수 (E3)(포인트)','목표주가','시가총액'], \
                               ['ADJprice', 'Consensus','TargetPrice','MarketValue'])


In [5]:
def CompuConsen(df):
    df['CompuConsen'] = 0
    Nowprice = df['총자산(천원)'] # 지금 주가
    Targetprice = df['총자산(평균)(천원)'] # 목표주가 
    for j in range(len(df) - 1):
        i = j+1
        if Nowprice[i]*1.4 <= Targetprice[i]:
            df['CompuConsen'][j] = 5
        elif ((Nowprice[i]*1.2 <= Targetprice[i]) | (Nowprice[i]*1.4 > Targetprice[i])) :
            df['CompuConsen'][j] = 4
        elif ((Nowprice[i]*0.8 <= Targetprice[i]) | (Nowprice[i]*1.2 > Targetprice[i])) :
            df['CompuConsen'][j] = 3
        elif (Nowprice[i]*0.8 > Targetprice[i]) :
            df['CompuConsen'][j] = 1
        else :
            df['CompuConsen'][j] = 0
    df['ConpuConsen'][j+1] = 0

In [77]:
CompuConsen(Onetable_kosdaq)